<center><img src="ImgProb02G03.png"></center>

In [1]:
import gmsh
import numpy as np
import matplotlib.pyplot as plt
import MEF as mef

In [2]:
lc = 0.1 #0.02
h = 0.3  # m
b = 0.6  # m
v = 0.3  # coef Poisson
Ei = 206e9  # Pa 
r = 0.03  # m  radio del agujero
t = 0.03  # m  espesor de la chapa
T = 10e6  # Pa

In [3]:
gmsh.initialize()
gmsh.model.add('Chapa_agujero_central')

In [4]:
pc = gmsh.model.geo.addPoint(b/2,0,0,lc)
p1 = gmsh.model.geo.addPoint(0,0,0,lc)
p2 = gmsh.model.geo.addPoint(b/2-r,0,0,lc/5)
p3 = gmsh.model.geo.addPoint(b/2,r,0,lc/100)
p4 = gmsh.model.geo.addPoint(b/2+r,0,0,lc/5)
p5 = gmsh.model.geo.addPoint(b,0,0,lc)
p6 = gmsh.model.geo.addPoint(b,h/2,0,lc)
p7 = gmsh.model.geo.addPoint(b/2,h/2,0,lc/100)
p8 = gmsh.model.geo.addPoint(0,h/2,0,lc)

l1 = gmsh.model.geo.addLine(p1,p2)
l2 = gmsh.model.geo.addLine(p4,p5)
l3 = gmsh.model.geo.addLine(p5,p6)  # traccionada
l4 = gmsh.model.geo.addLine(p6,p7)
l5 = gmsh.model.geo.addLine(p7,p8)
l6 = gmsh.model.geo.addLine(p8,p1)  # traccionada
c1 = gmsh.model.geo.addCircleArc(p2, pc, p3)
c2 = gmsh.model.geo.addCircleArc(p3, pc, p4)

C1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,c1,c2])

S1 = gmsh.model.geo.addPlaneSurface([C1])

In [5]:
gmsh.model.geo.synchronize()

In [6]:
Traccionado_pos = gmsh.model.addPhysicalGroup(1,[l3],)

gmsh.model.setPhysicalName(1,Traccionado_pos,'Traccionado_pos')
Traccionado_neg = gmsh.model.addPhysicalGroup(1,[l6])
gmsh.model.setPhysicalName(1,Traccionado_neg,'Traccionado_neg')
Empotrado_y = gmsh.model.addPhysicalGroup(1, [l1,l2])
gmsh.model.setPhysicalName(1,Empotrado_y,'Empotrado en y')
Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie,'Superficie')

gmsh.model.mesh.generate(2)
gmsh.model.geo.synchronize()
#gmsh.fltk.run()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Line)
Info    : [ 70%] Meshing curve 6 (Line)
Info    : [ 80%] Meshing curve 7 (Circle)
Info    : [ 90%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.00822201s, CPU 0.008594s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00522772s, CPU 0.00535s)
Info    : 349 nodes 703 elements


In [7]:
NodeInfo = gmsh.model.mesh.get_nodes()
Nn = NodeInfo[0].shape[0]
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)   # ETAGS = numeracion de cada elemento
                                                            # ELEMENTS = ravel de los nodos que posee cada elemento
MN = NodeInfo[1].reshape(Nn,3)  # matriz coordenadas de los nodos
MN = np.append(MN[1:,:],[MN[0,:]],0)  # llevo la primer fila al final porque nombra al nodo del centro como el último taggeado
MC = ELEMENTS.reshape(ETAGS.shape[0],3)  # matriz conectividad
Ne, Nnxe = MC.shape
MC = MC - np.ones([Ne,Nnxe])  # resto 1 para acomodar a lenguaje python

E = np.zeros(Ne)
for e in range(Ne):
    E[e] = Ei
    
glxn = 2
dimension_elementos = 2

gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
gmsh.write('chapa_agujero.msh')

Info    : Writing 'chapa_agujero.msh'...
Info    : Done writing 'chapa_agujero.msh'


In [8]:
nodos_empot_y, xyz_empot_y = gmsh.model.mesh.getNodesForPhysicalGroup(1, Empotrado_y)

In [9]:
s = []  # el vector s indica la direccion de que nodo sabemos cuanto se desplaza.
        # en este caso se asignan los valores de las direcciones de cada nodo empotrado
for n in nodos_empot_y-1:
    s.append(glxn*n+1)

s.append((NodeInfo[0][0]-1)*glxn)
s.append((NodeInfo[0][0]-1)*glxn+1)

s = np.array(s).astype(int)

Us = np.zeros([s.shape[0]]).reshape([-1,1])

r = mef.vector_complemento(s, MN, glxn).astype(int)

In [10]:
entityTraccionada_pos = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado_pos)
Tgroup_pos, Ttraccionada_pos, Ltraccionada_pos = gmsh.model.mesh.getElements(1, entityTraccionada_pos[0])
Nlineastraccionadas_pos = len(Ttraccionada_pos[0])
MCtraccionadas_pos = Ltraccionada_pos[0].reshape(Nlineastraccionadas_pos,2)-1

entityTraccionada_neg = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado_neg)
Tgroup_neg, Ttraccionada_neg, Ltraccionada_neg = gmsh.model.mesh.getElements(1, entityTraccionada_neg[0])
Nlineastraccionadas_neg = len(Ttraccionada_neg[0])
MCtraccionadas_neg = Ltraccionada_neg[0].reshape(Nlineastraccionadas_neg,2)-1

Fr = np.zeros_like(r)  # Vector fuerzas en los nodos que empotrados
for e in range(Nlineastraccionadas_pos):
    nodo1 = MCtraccionadas_pos[e,0].astype(int)
    nodo2 = MCtraccionadas_pos[e,1].astype(int)
    Ltraccionado = abs(MN[nodo2,1]-MN[nodo1,1])  # Longitud de la linea traccionada [e] que se esta analizando
    Fr[np.where(r == nodo1*2)[0][0]] += T*Ltraccionado*t/2  # Se distribuye la tensión aplicada según la longitud del elemento
    Fr[np.where(r == nodo2*2)[0][0]] += T*Ltraccionado*t/2
for e in range(Nlineastraccionadas_neg):
    nodo1 = MCtraccionadas_neg[e,0].astype(int)
    nodo2 = MCtraccionadas_neg[e,1].astype(int)
    Ltraccionado = abs(MN[nodo2,1]-MN[nodo1,1])  # Longitud de la linea traccionada [e] que se esta analizando
    Fr[np.where(r == nodo1*2)[0][0]] += -T*Ltraccionado*t/2  # Se distribuye la tensión aplicada según la longitud del elemento
    Fr[np.where(r == nodo2*2)[0][0]] += -T*Ltraccionado*t/2
Fr = Fr.reshape([-1,1])

In [11]:
K, Ke, D, B = mef.Kglobal_barra(MN, MC, E, glxn, dimension_elementos, v,t) 

F, U = mef.solve(K, r, s, Us, Fr)

sig = mef.tensiones(MC, U, D, B, glxn)
sig_xytau = np.hstack(list(sig.values()))

In [12]:
U2D = U.reshape(Nn,glxn)
U3D = np.append(U2D,np.zeros(U2D.shape[0]).reshape([-1,1]),1)

In [13]:
desps = gmsh.view.add("Desplazamientos")
Desps = gmsh.view.addModelData(desps, 0, 'Chapa_agujero_central', 'NodeData', NodeInfo[0]+1, U3D, numComponents=3)
gmsh.option.setNumber(f'View[{Desps}].VectorType',5)
gmsh.option.setNumber(f'View[{Desps}].GlyphLocation',2)

F2D = F.reshape(Nn,glxn)
F3D = np.append(F2D,np.zeros(F2D.shape[0]).reshape([-1,1]),1)

fuerzas = gmsh.view.add('Fuerzas')
Fuerzas = gmsh.view.addModelData(fuerzas, 0, 'Chapa_agujero_central','NodeData',NodeInfo[0]+1, F3D, numComponents=3)
#gmsh.option.setNumber(f'View[{fuerzas}].VectorType',4)
gmsh.option.setNumber(f'View[{Fuerzas}].VectorType',4)
#gmsh.option.setNumber(f'View[{fuerzas}].GlyphLocation',2)
gmsh.option.setNumber(f'View[{Fuerzas}].GlyphLocation',2)

sigx = gmsh.view.add('Tensiones en x')
sigy = gmsh.view.add('Tensiones en y')
sigxy = gmsh.view.add('Tensiones xy')
SIGX = gmsh.view.addModelData(sigx, 0, 'Chapa_agujero_central', 'ElementData', ETAGS, sig_xytau[0,:].reshape([-1,1]), numComponents=1)  # hago reshape porque tienen que ir vectores columnas
SIGY = gmsh.view.addModelData(sigy, 0, 'Chapa_agujero_central', 'ElementData', ETAGS, sig_xytau[1,:].reshape([-1,1]), numComponents=1)

In [14]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.10.5
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220701
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


<center><img src="FF_sin_fisura.png"></center>

$ \large
\text{Para medir la longitud de la fisura (aproximada): Se tienen en cuenta las coordenadas del primer elemento eliminado y del último.}
$

$ \large
\text{1ro: Creo una matriz que almacene los nodos de cada elemento eliminado}
$

$ \large
\text{2do: Considero que la fisura tiende a ser como una linea recta}
$

$ \large
\text{3ro: Obtengo la distancia entre dos puntos, un punto es un nodo del primer elemento eliminado, y el otro punto es un nodo del último elemento eliminado}
$

$ \large
\text{4to: Del primer elemento se toma el nodo de menor distancia en y. Del último elemento se toma el nodo con mayor distancia en y}
$

In [15]:
from tqdm.auto import tqdm

In [16]:
MCbroken = MC
ETAGSbroken = ETAGS
ElementosRotos = 43
MC_eliminados = np.zeros(3)
progress = tqdm(range(ElementosRotos))
for elrot in progress: #range(ElementosRotos):
    max_sig_element = np.where(sig_xytau[0,:] == sig_xytau[0,:].max())
    MC_eliminados = np.vstack([MC_eliminados,MCbroken[max_sig_element]])
    MCbroken = np.delete(MCbroken, max_sig_element, axis=0)
    ETAGSbroken = np.delete(ETAGSbroken, max_sig_element, axis=0)
    Ne, Nnxe = MCbroken.shape
    E = np.zeros(Ne)
    for e in range(Ne):
        E[e] = Ei
    K, Ke, D, B = mef.Kglobal_barra(MN, MCbroken, E, glxn, dimension_elementos, v,t)
    F, U = mef.solve(K, r, s, Us, Fr)
    sig = mef.tensiones(MCbroken, U, D, B, glxn)
    sig_xytau = np.hstack(list(sig.values()))  # saco las tensiones del diccionario, quedan en filas para cada elemento las tensiones x, y, xy
    U2D = U.reshape(Nn,glxn)
    U3D = np.append(U2D,np.zeros(U2D.shape[0]).reshape([-1,1]),1)
MC_eliminados = np.delete(MC_eliminados,0,0)

  0%|          | 0/43 [00:00<?, ?it/s]

LinAlgError: Singular matrix

In [ ]:
el1_nodos = MC_eliminados[0].astype(int)
el1_coord = np.array([[MN[el1_nodos[0]]],[MN[el1_nodos[1]]],[MN[el1_nodos[2]]]])

el2_nodos = MC_eliminados[-1].astype(int)
el2_coord = np.array([[MN[el2_nodos[0]]],[MN[el2_nodos[1]]],[MN[el2_nodos[2]]]])

cd_in = el1_coord[np.where(el1_coord == el1_coord[:,0][:,1].max())[0][0]].tolist()[0]  # coord del nodo donde inicia la fisura
cd_p = el2_coord[np.where(el2_coord == el2_coord[:,0][:,1].max())[0][0]].tolist()[0]  # coord del nodo en la punta de la fisura

longitud_fisura = ((cd_p[0]-cd_in[0])**2+(cd_p[1]-cd_in[1])**2)**(1/2)*1e3  # en mm

In [ ]:
print(f'\033[30m \033[107m Longitud de la fisura = {longitud_fisura} mm')

In [ ]:
desps = gmsh.view.add("Desplazamientos")
Desps = gmsh.view.addModelData(desps, 0, 'Chapa_agujero_central', 'NodeData', NodeInfo[0]+1, U3D, numComponents=3)
gmsh.option.setNumber(f'View[{desps}].VectorType',5)

F2D = F.reshape(Nn,glxn)
F3D = np.append(F2D,np.zeros(F2D.shape[0]).reshape([-1,1]),1)

fuerzas = gmsh.view.add('Fuerzas')
Fuerzas = gmsh.view.addModelData(fuerzas, 0, 'Chapa_agujero_central','NodeData',NodeInfo[0]+1, F3D, numComponents=3)
gmsh.option.setNumber(f'View[{fuerzas}].VectorType',4)
gmsh.option.setNumber(f'View[{fuerzas}].GlyphLocation',2)

sigx = gmsh.view.add('Tensiones en x')
sigy = gmsh.view.add('Tensiones en y')
sigxy = gmsh.view.add('Tensiones xy')
SIGX = gmsh.view.addModelData(sigx, 0, 'Chapa_agujero_central', 'ElementData', ETAGSbroken, sig_xytau[0,:].reshape([-1,1]), numComponents=1)  # hago reshape porque tienen que ir vectores columnas
SIGY = gmsh.view.addModelData(sigy, 0, 'Chapa_agujero_central', 'ElementData', ETAGSbroken, sig_xytau[1,:].reshape([-1,1]), numComponents=1)

gmsh.fltk.run()

<center><img src="FFisura_Hermosa(1).png"></center>

<center><img src="FFisura_Hermosa(zoom)(1).png"></center>

<div style="background:#999900"> Buen'isimo Luciano ! está muy bien ! haría un par de modificaciones para mejorarlo:
    
  <ol>
      <li> en vez de usar las tensiones $\sigma = D \varepsilon$ usaría las tensiones principales, fijate en la <a url="https://isabatoingenieria.com.ar/moodle/pluginfile.php/2466/mod_resource/content/10/Clase-7.pdf">Clase 7 de Rubén</a>, pagina 12. fijate que el elmemento de máxima tensión debería ser el que tiene la punta sobre la fisura. Lo podremos reproducir así ?
      <li> No estoy seguro de cuál es el criterio de elección del elemento a marcar, te fijaste bien en el capítulo de la tesis? en cualquier caso la tensión máxima está bien para tener algo ahora.
      <li> fijate que los elementos se te van agrandando a medida que avanzas. eso es porque el factor de escala del punto que está en el borde es mas grande que el del agujero. fijate si se lo podes achicar.
       <li> en vez de sacar el elemento, se puede hacer un marcador:
           
           $ Marcador_0  = 1 \forall e $
           
           luego:
           
           $ Marcador_t ^i = -1 if \sigma (t) _i > \sigma_{critico} $ ( parecido a lo que hace el espa~nol de la tesis)
          
           con eso se podría hacer la peliculita del avance de la fisura
      <li> ya para lujo: cuando en la punta de la fisura se cumple el criterio del dieter (de orowan ? el del $K_{IC}$) tendrías que darle  rotura catastrófica. eso te daría una idea de la máxima fisura que podes aguantar.
    </ol>
 </div>

<div style="background:#999900"> 
despues, como veras en este notebook, trat'e de cambiar el factor de escala para forzar los elementos sobre la fisura. en un momento la matriz empieza a dar sinuglar. eso quiere decir que hay que revisar las condiciones de contorno!
</div>